In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import copy

import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook', font_scale=1.2)

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

from IPython.display import display

In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [3]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser, host='/var/run/postgresql')
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [4]:
query = """
SELECT * FROM icu_features
"""
ef = pd.read_sql_query(query, con)
ef.head()

,icustay_id,hadm_id,subject_id,age,gender,height,weight,ethnicity,insurance,filter_vaso,...,pc_bronch,pc_cath,pc_echo,pc_pressor,pc_rhc,pc_thora,pc_vent,passed_filters,use_record,bmi
0,200001,152234,55973,22290 days 19:06:12,F,167.851667,27.669135,ASIAN - ASIAN INDIAN,Medicare,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,9.820741
1,200003,163557,27513,17625 days 19:50:04,M,177.800000,78.224998,WHITE,Private,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,24.744692
2,200006,189514,10950,19736 days 11:28:14,M,165.100000,82.400002,OTHER,Medicaid,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.229647
3,200007,129310,20707,15818 days 10:03:37,M,177.800000,126.000000,WHITE,Private,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.857223
4,200009,129607,29904,17353 days 10:34:32,F,160.020000,85.833331,WHITE,Private,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,33.520264


In [10]:
len(ef)

61532

In [11]:
F = (ef['use_record'] == 1)

In [12]:
ef = ef.loc[F]
len(ef)

9320

In [13]:
ef.ea_lv_systolic.value_counts()

 0.0    2297
-1.0     308
 3.0     220
 1.0     183
 2.0     147
-2.0      71
-3.0      30
Name: ea_lv_systolic, dtype: int64

In [14]:
ef['has_echo'] = ~np.isnan(ef.ea_key)

In [16]:
ef.ea_lv_systolic.fillna(value=-99, inplace=True)

In [19]:
ef.groupby(['has_echo', 'ea_lv_systolic']).apply(len)

has_echo  ea_lv_systolic
False     -99.0             5634
True      -99.0              430
          -3.0                30
          -2.0                71
          -1.0               308
           0.0              2297
           1.0               183
           2.0               147
           3.0               220
dtype: int64

In [32]:
ef.groupby('has_echo').apply(lambda x: x.gender.value_counts())

gender,M,F
has_echo,,
False,2935,2699
True,1920,1766


In [33]:
ef.groupby('has_echo').apply(lambda x: x.gender.value_counts(normalize=True))

gender,M,F
has_echo,,
False,0.520944,0.479056
True,0.520890,0.479110


In [24]:
ef.groupby(['has_echo', 'ea_lv_systolic']).apply(lambda x: x.gender.value_counts()).unstack(2)

F     M
has_echo ea_lv_systolic            
False    -99.0           2699  2935
True     -99.0            175   255
         -3.0               9    21
         -2.0              30    41
         -1.0             169   139
          0.0            1152  1145
          1.0              76   107
          2.0              67    80
          3.0              88   132

In [25]:
ef.groupby(['has_echo', 'ea_lv_systolic']).apply(lambda x: x.gender.value_counts(normalize=True)).unstack(2)

F         M
has_echo ea_lv_systolic                    
False    -99.0           0.479056  0.520944
True     -99.0           0.406977  0.593023
         -3.0            0.300000  0.700000
         -2.0            0.422535  0.577465
         -1.0            0.548701  0.451299
          0.0            0.501524  0.498476
          1.0            0.415301  0.584699
          2.0            0.455782  0.544218
          3.0            0.400000  0.600000

In [38]:
def subgroup(g): 
    display(ef.groupby('has_echo').apply(lambda x: x[g].value_counts()))
    display(ef.groupby('has_echo').apply(lambda x: x[g].value_counts(normalize=True)))
    display(ef.groupby(['has_echo', 'ea_lv_systolic']).apply(lambda x: x[g].value_counts()).unstack(2))
    display(ef.groupby(['has_echo', 'ea_lv_systolic']).apply(lambda x: x[g].value_counts(normalize=True)).unstack(2))
    
    
    

In [35]:
subgroup('gender')

gender,M,F
has_echo,,
False,2935,2699
True,1920,1766


gender,M,F
has_echo,,
False,0.520944,0.479056
True,0.520890,0.479110


F     M
has_echo ea_lv_systolic            
False    -99.0           2699  2935
True     -99.0            175   255
         -3.0               9    21
         -2.0              30    41
         -1.0             169   139
          0.0            1152  1145
          1.0              76   107
          2.0              67    80
          3.0              88   132

F         M
has_echo ea_lv_systolic                    
False    -99.0           0.479056  0.520944
True     -99.0           0.406977  0.593023
         -3.0            0.300000  0.700000
         -2.0            0.422535  0.577465
         -1.0            0.548701  0.451299
          0.0            0.501524  0.498476
          1.0            0.415301  0.584699
          2.0            0.455782  0.544218
          3.0            0.400000  0.600000

In [37]:
subgroup('ex_congestive_heart_failure')

ex_congestive_heart_failure,0,1
has_echo,,
False,4369,1265
True,2369,1317


ex_congestive_heart_failure,0,1
has_echo,,
False,0.775470,0.224530
True,0.642702,0.357298


0     1
has_echo ea_lv_systolic            
False    -99.0           4369  1265
True     -99.0            230   200
         -3.0              17    13
         -2.0              31    40
         -1.0             229    79
          0.0            1639   658
          1.0              90    93
          2.0              58    89
          3.0              75   145

0         1
has_echo ea_lv_systolic                    
False    -99.0           0.775470  0.224530
True     -99.0           0.534884  0.465116
         -3.0            0.566667  0.433333
         -2.0            0.436620  0.563380
         -1.0            0.743506  0.256494
          0.0            0.713539  0.286461
          1.0            0.491803  0.508197
          2.0            0.394558  0.605442
          3.0            0.340909  0.659091

In [169]:
def subgroup(g):
    from scipy.stats import fisher_exact, chi2_contingency
    
   
        
    
    counts = ef.groupby(g).apply(lambda x: x.mortality.value_counts())
    if type(counts) is pd.Series:
        counts = counts.unstack(-1).fillna(value=0)
    marginal = counts.sum(axis=0)
    #if type(g) is list:
    #    counts = counts.unstack(-1)
        
    display(counts)
    
    marginal = float(marginal.loc[True])/marginal.sum()
    proportions = counts.div(counts.sum(axis=1), axis=0)
    proportions['fold_change'] = proportions[True]/marginal
    
    if len(proportions) > 2:
        pval_array = np.zeros(len(proportions))
        for i in range(len(proportions)):
            this = counts.iloc[i]
            rest = counts.sum() - this
            table = pd.DataFrame([this, rest])
            _, pval = fisher_exact(table)
            pval_array[i] = pval
        signif = (pval_array < 0.05)
        proportions['p-value'] = pval_array
        proportions['significant @ p=0.05'] = signif
        proportions['support'] = counts.sum(axis=1)
        
    
    display(proportions)
    #display(counts/counts.sum(axis=1))
    
    if len(counts) == 2:
        _, pval = fisher_exact(counts)
        print('Fisher\'s exact p-value:', pval)
    elif len(counts) > 2:
        if np.any(counts < 5):
            print('Warning! Chi2 not advised if any count < 5')
        _, pval, _, _ = chi2_contingency(counts)
        print('Chi2 p-value:', pval)
    
    ## echo vs no echo
    #counts = ef.groupby('has_echo').apply(lambda x: x[g].value_counts())
    
def subgroup_fluids(g):
    from scipy.stats import ttest_ind
    
    def fluid_analysis(x):
        F = ~(x.fb_day1_balance_truncated == 1)
        fl_live = x.fb_day1_balance_ml.loc[~x.mortality & F]
        fl_live = fl_live.loc[~np.isnan(fl_live)]
        fl_dead = x.fb_day1_balance_ml.loc[x.mortality & F]
        fl_dead = fl_dead.loc[~np.isnan(fl_dead)]
        delta = fl_live.mean() - fl_dead.mean()
        _, pval = ttest_ind(fl_live, fl_dead, equal_var=False)
        signif = (pval < 0.05)
        return pd.Series({'fluid_day1_diff': delta, 'p-value': pval
                          , 'significant @ p=0.05': signif
                          , 'support': len(fl_live)+len(fl_dead)})
    
    if len(g) == 0:
        table = pd.DataFrame([fluid_analysis(ef)])
    else:
        table = ef.groupby(g).apply(fluid_analysis)
    display(table)
    
    

What we want to understand:

+ _p(mortality | $lv_{mild}$, fluids) = p($lv_{mild}$, fluids | mortality)p(mortality)/p($lv_{mild}$, fluids)_
+ Is mortality conditionaly independent of fluids and $lv_{mild}$ given some other variable?
  + find _c_ s.t. _p(mortality, $lv_{mild}$, fluids | c) = p(mortality | c)p($lv_{mild}$, fluids | c)_

In [170]:
tod = ef.dod - ef.intime
ef['mortality'] = tod < np.timedelta64(30, 'D')

In [171]:
subgroup_fluids([])

,fluid_day1_diff,p-value,significant @ p=0.05,support
0,-686.52083,1.103885e-28,True,8329


In [172]:
subgroup('has_echo')

mortality,False,True
has_echo,,
False,4103,1531
True,2597,1089


mortality,False,True,fold_change
has_echo,,,
False,0.728257,0.271743,0.966658
True,0.704558,0.295442,1.050962


Fisher's exact p-value: 0.0133556149149


In [173]:
subgroup_fluids('has_echo')

,fluid_day1_diff,p-value,significant @ p=0.05,support
has_echo,,,,
False,-803.860637,8.311518e-25,True,5022
True,-504.635392,5.009536e-07,True,3307


In [174]:
subgroup(['has_echo', 'ea_lv_systolic'])

mortality                False  True 
has_echo ea_lv_systolic              
False    -99.0            4103   1531
True     -99.0             295    135
         -3.0               19     11
         -2.0               38     33
         -1.0              184    124
          0.0             1689    608
          1.0              137     46
          2.0               98     49
          3.0              137     83

mortality                   False      True  fold_change   p-value  \
has_echo ea_lv_systolic                                              
False    -99.0           0.728257  0.271743     0.966658  0.013356   
True     -99.0           0.686047  0.313953     1.116812  0.124196   
         -3.0            0.633333  0.366667     1.304326  0.310949   
         -2.0            0.535211  0.464789     1.653371  0.001252   
         -1.0            0.597403  0.402597     1.432140  0.000003   
          0.0            0.735307  0.264693     0.941580  0.044952   
          1.0            0.748634  0.251366     0.894173  0.406485   
          2.0            0.666667  0.333333     1.185751  0.165373   
          3.0            0.622727  0.377273     1.342054  0.001804   

mortality               significant @ p=0.05  support  
has_echo ea_lv_systolic                                
False    -99.0                          True     5634  
True     -99.0                         False      430  
         -3.0                          False       30  
         -2.0                           True       71  
         -1.0                           True      308  
          0.0                           True     2297  
          1.0                          False      183  
          2.0                          False      147  
          3.0                           True      220

Chi2 p-value: 2.70890008027e-09


In [175]:
subgroup_fluids(['has_echo', 'ea_lv_systolic'])

fluid_day1_diff       p-value significant @ p=0.05  \
has_echo ea_lv_systolic                                                       
False    -99.0               -803.860637  8.311518e-25                 True   
True     -99.0               -124.767457  6.280460e-01                False   
         -3.0                -517.441453  6.336798e-01                False   
         -2.0                -371.720469  5.338452e-01                False   
         -1.0                -992.137347  2.947838e-03                 True   
          0.0                -472.823157  3.236973e-04                 True   
          1.0                 268.645489  5.314346e-01                False   
          2.0                -751.056420  1.987013e-01                False   
          3.0               -1191.935608  3.506012e-03                 True   

                         support  
has_echo ea_lv_systolic           
False    -99.0              5022  
True     -99.0               383  
         -3.0                 25  
         -2.0                 59  
         -1.0                268  
          0.0               2088  
          1.0                162  
          2.0                132  
          3.0                190

In [176]:
ef.ea_rv_cavity.fillna(value=-99, inplace=True)

In [177]:
subgroup(['has_echo', 'ea_rv_cavity'])

False   True 
has_echo ea_rv_cavity                
False    -99.0         4103.0  1531.0
True     -99.0          199.0   106.0
         -50.0            0.0     1.0
         -3.0           115.0    48.0
         -2.0           135.0    92.0
         -1.0             5.0     4.0
          0.0          1811.0   679.0
          1.0           221.0    90.0
          2.0           111.0    69.0

False      True  fold_change   p-value  \
has_echo ea_rv_cavity                                              
False    -99.0         0.728257  0.271743     0.966658  0.013356   
True     -99.0         0.652459  0.347541     1.236291  0.009595   
         -50.0         0.000000  1.000000     3.557252  0.281116   
         -3.0          0.705521  0.294479     1.047534  0.725222   
         -2.0          0.594714  0.405286     1.441706  0.000049   
         -1.0          0.555556  0.444444     1.581001  0.279519   
          0.0          0.727309  0.272691     0.970030  0.285731   
          1.0          0.710611  0.289389     1.029430  0.748510   
          2.0          0.616667  0.383333     1.363613  0.003247   

                      significant @ p=0.05  support  
has_echo ea_rv_cavity                                
False    -99.0                        True   5634.0  
True     -99.0                        True    305.0  
         -50.0                       False      1.0  
         -3.0                        False    163.0  
         -2.0                         True    227.0  
         -1.0                        False      9.0  
          0.0                        False   2490.0  
          1.0                        False    311.0  
          2.0                         True    180.0

Warning! Chi2 not advised if any count < 5
Chi2 p-value: 2.47318228119e-06
